Evolution under Hamiltonian of Heisenberg interaction in an external field, nearest neighbor line

We are choosing a circuit that has a linear connectivity map which is the simplest possible. The implemented circuit uses a lot of single qubit gates to implement spin interaction along the different axes and the goal is to see whether a custom compiler does better at reducing these gates than the default one.

In [19]:
from qiskit_ibm_runtime.fake_provider import FakeTorino
from qiskit import QuantumCircuit

backend = FakeTorino()

import time
import numpy as np

Define circuit

In [20]:
num_qubits = 20
num_layers = 3
qc=QuantumCircuit(num_qubits)
for i_layer in range(num_layers):
    for i in range(num_qubits-1):    
        j = (i + 1)
    
        qc.rz(np.random.rand(), i)
        qc.rz(np.random.rand(), j)
        
        qc.cx(i, j)
        qc.rz(0.1, j)
        qc.cx(i, j)
    
        qc.h(i)
        qc.h(j)
        qc.cx(i, j)
        qc.rz(0.1, j)
        qc.cx(i, j)
        qc.h(i)
        qc.h(j)
    
        qc.ry(np.pi/2, i)
        qc.ry(np.pi/2, j)
        qc.cx(i, j)
        qc.rz(0.1, j)
        qc.cx(i, j)
        qc.ry(-np.pi/2, i)
        qc.ry(-np.pi/2, j)
# Get gate counts
gate_counts = qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))

OrderedDict({'cx': 342, 'rz': 285, 'h': 228, 'ry': 228})
Number of 2-qubit gates:  342
Number of 1-qubit gates:  285


Optionally print out the circuit

In [21]:
# qc.draw()

Compile with default passes

In [22]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
default_pm = generate_preset_pass_manager(backend=backend, optimization_level=3)

t1 = time.time()
default_qc = default_pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))

Time taken:  0.7366163730621338
OrderedDict({'sx': 342, 'rz': 331, 'cz': 114, 'x': 5})
Number of 2-qubit gates:  114
Number of 1-qubit gates:  678


Create custom compiler

In [23]:
from qiskit.transpiler import PassManager
import qiskit.transpiler.passes as passes 


custom_pm = PassManager()

custom_pm.append(passes.Optimize1qGatesDecomposition())

custom_pm.append(passes.Collect2qBlocks())
custom_pm.append(passes.ConsolidateBlocks())

# custom_pm.append(passes.Decompose())
# custom_pm.append(passes.Optimize1qGates())

custom_qc = custom_pm.run(qc)

# Get gate counts
gate_counts = custom_qc.count_ops()
print(gate_counts)

# custom_qc.draw()

OrderedDict({'unitary': 57})


In [24]:
custom_qc = default_pm.run(custom_qc)

# Get gate counts
gate_counts = custom_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0))
print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))


OrderedDict({'sx': 342, 'rz': 325, 'cz': 114, 'x': 7})
Number of 2-qubit gates:  114
Number of 1-qubit gates:  674
